In [1]:
%pwd


'/home/ml01/MLOps/hugginface_project/Text-Summarizer-MLOPS/research'

In [2]:
import os
os.chdir("../")
%pwd

'/home/ml01/MLOps/hugginface_project/Text-Summarizer-MLOPS'

In [11]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass
class ModelTrainerConfig:
    #these are from config.yaml
    root_dir: Path 
    data_path: Path 
    model_ckpt: Path

    #these are from params.yaml
    num_train_epochs: int 
    warmup_steps: int 
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float 
    logging_steps: int 
    evaluation_strategy: str  
    eval_steps: int 
    save_steps: int
    gradient_accumulation_steps: int 

In [12]:
from src.text_summarizer.utils.common import read_yaml, create_directories
from src.text_summarizer.constants import *

class ConfigurationManager:
    def __init__(self, config_path= CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps,
            save_steps=params.save_steps
        )

        return model_trainer_config


In [13]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer 
from transformers import DataCollatorForSeq2Seq 
import torch 

from datasets import load_from_disk

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config 
    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model= model_pegasus)

        #loading the data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, 
            num_train_epochs=self.config.num_train_epochs, 
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, 
            per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            eval_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])
        
        # trainer.train()

        ##save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))

        ##save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))


In [14]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer_config = ModelTrainer(config=model_trainer_config)
model_trainer_config.train()

[2025-06-26 17:27:42,556: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-06-26 17:27:42,559: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-26 17:27:42,559: INFO: common: created directory at: artifacts]
[2025-06-26 17:27:42,560: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[2025-06-26 17:27:51,838] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/tmp/ipykernel_334775/2725061035.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,


[2025-06-26 17:27:51,976: INFO: spawn: x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -c /tmp/tmp1ipzz5gb/test.c -o /tmp/tmp1ipzz5gb/test.o]
[2025-06-26 17:27:52,039: INFO: spawn: x86_64-linux-gnu-gcc /tmp/tmp1ipzz5gb/test.o -laio -o /tmp/tmp1ipzz5gb/a.out]


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


[2025-06-26 17:27:52,337: INFO: spawn: x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -c /tmp/tmpsu9j8iu0/test.c -o /tmp/tmpsu9j8iu0/test.o]
[2025-06-26 17:27:52,349: INFO: spawn: x86_64-linux-gnu-gcc /tmp/tmpsu9j8iu0/test.o -L/usr/local/cuda -L/usr/local/cuda/lib64 -lcufile -o /tmp/tmpsu9j8iu0/a.out]
[2025-06-26 17:27:52,411: INFO: spawn: x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -c /tmp/tmphnl66m28/test.c -o /tmp/tmphnl66m28/test.o]
[2025-06-26 17:27:52,424: INFO: spawn: x86_64-linux-gnu-gcc /tmp/tmphnl66m28/test.o -laio -o /tmp/tmphnl66m28/a.out]


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


[2025-06-26 17:27:53,256] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False


/home/ml01/.virtualenvs/mlops_huggingface/lib/python3.10/site-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
